In [ ]:
import pandas as pd


train = pd.read_csv("/content/drive/MyDrive/Implicit/Epinions/trainImplicit.csv")
train = train[["userId","productId","rating"]]
train

userId  productId  rating
0        37909      65269       1
1        11562      45908       1
2        30899       2214       1
3        31710      69277       1
4          121       8449       1
...        ...        ...     ...
396309   36865      40629       1
396310    8110       4877       1
396311   32154      40691       1
396312   36993     106422       1
396313    1947       4702       1

[396314 rows x 3 columns]

In [ ]:
print(max(train["userId"]))
print(max(train["productId"]))

37951
119014


In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
mat_data = csc_matrix((37952,119015), dtype=np.int8)
mat_data = mat_data.tolil()
print(mat_data.shape)
for uid,pid,_ in train.values:
  mat_data[uid,pid] = 1
#mat_data = mat_data.tocoo()

(37952, 119015)


In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Epinions/Data/train_popularity_1.txt")

In [ ]:
import pickle

with open('/content/drive/MyDrive/Implicit/Epinions/Data/user_neighbor_items_test.pickle', 'rb') as handle:
    user_neighbor_items = pickle.load(handle)

with open('/content/drive/MyDrive/Implicit/Epinions/Data/user_neighbor_popularity_test.pickle', 'rb') as handle:
    user_neighbors_popularity = pickle.load(handle)

In [ ]:
mat_data_coo = mat_data.tocoo(copy=True)
mat_data_lil = mat_data_coo.tolil(copy=True)
mat_data_todok = mat_data_coo.todok(copy=True)

In [ ]:
class Sampler(object):

    def __init__(self, lil_record, dok_record, neg_sample_rate,num_user,num_item):

        self.lil_record = lil_record
        self.record = list(dok_record.keys())
        self.neg_sample_rate = neg_sample_rate
        self.n_user = num_user
        self.n_item = num_item
    
    def sample(self, index, **kwargs):

        raise NotImplementedError
    
    def get_pos_user_item(self, index):

        user = self.record[index][0]
        pos_item = self.record[index][1]

        return user, pos_item
    
    def generate_negative_samples(self, user, **kwargs):

        negative_samples = np.full(self.neg_sample_rate, -1, dtype=np.int64)

        user_pos = self.lil_record.rows[user]
        for count in range(self.neg_sample_rate):

            item = np.random.randint(self.n_item)
            while item in user_pos or item in negative_samples:
                item = np.random.randint(self.n_item)
            negative_samples[count] = item
        
        return negative_samples

In [ ]:
class SIDRSampler(Sampler):

    def __init__(self, lil_record, dok_record, neg_sample_rate, popularity,user_neighbors_popularity,neighbor_items, all_items, margin=10, pool=10):

        super(SIDRSampler, self).__init__(lil_record, dok_record, neg_sample_rate,37952,119015)
        self.popularity = popularity
        self.user_neighbors_popularity = user_neighbors_popularity
        self.neighbor_items = neighbor_items
        self.margin = margin
        self.all_items = all_items
        self.pool = pool

    def adapt(self, epoch, decay):

        self.margin = self.margin*decay

    def generate_negative_neighbor_samples(self, user, pos_item):
        negative_samples = np.full(self.neg_sample_rate, -1, dtype=np.int64)
        nei_mask_type = np.full(self.neg_sample_rate, False, dtype=np.bool)
        mask_type = np.full(self.neg_sample_rate, False, dtype=np.bool)

        user_pos = self.lil_record.rows[user]
        weights = self.user_neighbors_popularity[user]

        nei_items = np.array(self.neighbor_items[user])
        nei_items = nei_items[np.logical_not(np.isin(nei_items,user_pos))]
        num_nei_items = len(nei_items)
        item_pos_pop = self.popularity[pos_item]

        pop_items = np.nonzero(self.popularity > item_pos_pop + self.margin)[0]
        pop_items = pop_items[np.logical_not(np.isin(pop_items, user_pos))]
        num_pop_items = len(pop_items)

        unpop_items = np.nonzero(self.popularity < item_pos_pop - 10)[0]
        unpop_items = np.nonzero(self.popularity < item_pos_pop/2)[0]
        unpop_items = unpop_items[np.logical_not(np.isin(unpop_items, user_pos))]
        num_unpop_items = len(unpop_items)

        if num_nei_items < self.pool:
          if num_pop_items < self.pool:
            for count in range(self.neg_sample_rate):

                index = np.random.randint(num_unpop_items)
                item = unpop_items[index]
                while item in negative_samples:
                    index = np.random.randint(num_unpop_items)
                    item = unpop_items[index]

                negative_samples[count] = item
                mask_type[count] = False
                nei_mask_type[count] = False
            
          elif num_unpop_items < self.pool:  
            for count in range(self.neg_sample_rate):
                index = np.random.randint(num_pop_items)
                item = pop_items[index]
                while item in negative_samples:
                    index = np.random.randint(num_pop_items)
                    item = pop_items[index]

                negative_samples[count] = item
                mask_type[count] = True
                nei_mask_type[count] = False        
          else:
            for count in range(self.neg_sample_rate):
                if np.random.random() < 0.5:
                    index = np.random.randint(num_pop_items)
                    item = pop_items[index]
                    while item in negative_samples:
                        index = np.random.randint(num_pop_items)
                        item = pop_items[index]

                    negative_samples[count] = item
                    mask_type[count] = True
                    nei_mask_type[count] = False
                else:
                    index = np.random.randint(num_unpop_items)
                    item = unpop_items[index]
                    while item in negative_samples:
                        index = np.random.randint(num_unpop_items)
                        item = unpop_items[index]

                    negative_samples[count] = item
                    mask_type[count] = False
                    nei_mask_type[count] = False
        else:
          for count in range(self.neg_sample_rate):
                #if np.random.random() < 0.5:
                    index = np.random.choice(num_nei_items,1,weights)
                    item = nei_items[index]
                    while item in negative_samples:
                        index = np.random.choice(num_nei_items,1,weights)
                        item = nei_items[index]

                    negative_samples[count] = item
                    mask_type[count] = False
                    nei_mask_type[count] = True
        return negative_samples,mask_type,nei_mask_type            
    def sample(self, index):

        user, pos_item = self.get_pos_user_item(index)

        users = np.full(self.neg_sample_rate, user, dtype=np.int64)
        items_pos = np.full(self.neg_sample_rate, pos_item, dtype=np.int64)
        items_neg, mask_type, nei_mask_type, = self.generate_negative_neighbor_samples(user, pos_item=pos_item)

        return users, items_pos, items_neg, mask_type, nei_mask_type

In [ ]:
from torch.utils.data import Dataset, DataLoader

class FactorizationDataset(Dataset):

    def __init__(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,neighbor_items,all_items):
        self.make_sampler(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,neighbor_items,all_items)

    def make_sampler(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity):
        train_coo_record = mat_data_coo
        train_lil_record = mat_data_lil
        train_dok_record = mat_data_todok

        self.make_sampler_core(train_lil_record, train_dok_record)

    def __len__(self):

        return len(self.sampler.record)

    def __getitem__(self, index):

        raise NotImplementedError

class SIDRFactorizationDataset(FactorizationDataset):

    def __init__(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,neighbor_items,all_items):

        super(SIDRFactorizationDataset, self).__init__(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,neighbor_items,all_items)

    def make_sampler(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,neighbor_items,all_items):

        self.popularity = train_popularity

        train_coo_record = mat_data_coo
        train_lil_record = mat_data_lil
        train_dok_record = mat_data_todok

        self.sampler = SIDRSampler(train_lil_record, train_dok_record, 6, train_popularity,user_neighbors_popularity,neighbor_items,all_items)

    def __len__(self):

        return len(self.sampler.record)

    def __getitem__(self, index):

        if index < len(self.sampler.record):
            users, items_p, items_n, mask, nei_mask = self.sampler.sample(index)
            mask = torch.BoolTensor(mask)
            nei_mask = torch.BoolTensor(nei_mask)
        
        return users, items_p, items_n, mask, nei_mask

    def adapt(self, epoch, decay):
        self.sampler.adapt(epoch, decay)

In [ ]:
dataset = SIDRFactorizationDataset(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity,user_neighbors_popularity,user_neighbor_items,train["productId"].unique())
x = DataLoader(dataset, batch_size=512, shuffle=True, drop_last=False)

In [ ]:
#!pip install dgl
!pip install dgl-cu102 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [ ]:
import scipy.sparse
import math
import numpy as np

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import dgl.function as fn
import dgl


from tqdm import tqdm

import random

class LGConv(nn.Module):

    def __init__(self,
                 in_feats,
                 out_feats,
                 k=1,
                 cached=False,
                 bias=True,
                 norm=None):
        super(LGConv, self).__init__()
        self._cached = cached
        self._cached_h = None
        self._k = k
        self.norm = norm

    def forward(self, graph, feat):

        graph = graph.local_var()
        if self._cached_h is not None:
            feat = self._cached_h
        else:
            # compute normalization
            degs = graph.in_degrees().float().clamp(min=1)
            norm = torch.pow(degs, -0.5)
            norm = norm.to(feat.device).unsqueeze(1)
            # compute (D^-1 A^k D)^k X
            for _ in range(self._k):
                feat = feat * norm
                graph.ndata['h'] = feat
                graph.update_all(fn.copy_u('h', 'm'),
                                 fn.sum('m', 'h'))
                feat = graph.ndata.pop('h')
                feat = feat * norm

            if self.norm is not None:
                feat = self.norm(feat)

            # cache feature
            if self._cached:
                self._cached_h = feat

        return feat

class LGNSIDR(nn.Module):

    def __init__(self, num_users, num_items, embedding_size, num_layers, dropout, dis_pen, int_weight, pop_weight,soc_weight,graph_path):

        super(LGNSIDR, self).__init__()

        self.n_user = num_users
        self.n_item = num_items

        self.int_weight = int_weight
        self.pop_weight = pop_weight
        self.soc_weight = soc_weight

        self.embeddings_int = Parameter(torch.FloatTensor(num_users + num_items, embedding_size))
        self.embeddings_pop = Parameter(torch.FloatTensor(num_users + num_items, embedding_size))
        self.embeddings_soc = Parameter(torch.FloatTensor(num_users + num_items, embedding_size))
        self.init_graph(graph_path)

        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(LGConv(embedding_size, embedding_size, 1))

        self.dropout = dropout

        self.criterion_discrepancy = self.calculate_disc_loss
        #self.criterion_discrepancy = self.dcor

        self.dis_pen = dis_pen

        self.init_params()

    def init_graph(self, graph_path):

        self.coo_adj_graph = scipy.sparse.load_npz(graph_path)
        self.graph = dgl.DGLGraph()

        num_nodes = self.coo_adj_graph.shape[0]
        self.graph.add_nodes(num_nodes)
        self.graph.ndata['feature'] = torch.arange(num_nodes)

        self.graph.add_edges(self.coo_adj_graph.row, self.coo_adj_graph.col)
        self.graph.add_edges(self.graph.nodes(), self.graph.nodes())

        print(self.graph.nodes())
        #self.graph.readonly()

    def dcor(self, x, y):

        a = torch.norm(x[:,None] - x, p = 2, dim = 2)
        b = torch.norm(y[:,None] - y, p = 2, dim = 2)

        A = a - a.mean(dim=0)[None,:] - a.mean(dim=1)[:,None] + a.mean()
        B = b - b.mean(dim=0)[None,:] - b.mean(dim=1)[:,None] + b.mean() 

        n = x.size(0)

        dcov2_xy = (A * B).sum()/float(n * n)
        dcov2_xx = (A * A).sum()/float(n * n)
        dcov2_yy = (B * B).sum()/float(n * n)
        dcor = -torch.sqrt(dcov2_xy)/torch.sqrt(torch.sqrt(dcov2_xx) * torch.sqrt(dcov2_yy))

        return dcor

    def calculate_disc_loss(self,x, y, kernel="rbf"):
      """Emprical maximum mean discrepancy. The lower the result
       the more evidence that distributions are the same.

      Args:
          x: first sample, distribution P
          y: second sample, distribution Q
          kernel: kernel type such as "multiscale" or "rbf"
      """
      xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
      rx = (xx.diag().unsqueeze(0).expand_as(xx))
      ry = (yy.diag().unsqueeze(0).expand_as(yy))
      
      dxx = rx.t() + rx - 2. * xx # Used for A in (1)
      dyy = ry.t() + ry - 2. * yy # Used for B in (1)
      dxy = rx.t() + ry - 2. * zz # Used for C in (1)
      
      
      XX, YY, XY = (torch.zeros(xx.shape).cuda(),
                    torch.zeros(xx.shape).cuda(),
                    torch.zeros(xx.shape).cuda())
      '''
      XX, YY, XY = (torch.zeros(xx.shape),
                    torch.zeros(xx.shape),
                    torch.zeros(xx.shape))
      '''
      
      if kernel == "multiscale":
        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
          XX += a**2 * (a**2 + dxx)**-1
          YY += a**2 * (a**2 + dyy)**-1
          XY += a**2 * (a**2 + dxy)**-1    
      elif kernel == "rbf":  
        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
          XX += torch.exp(-0.5*dxx/a)
          YY += torch.exp(-0.5*dyy/a)
          XY += torch.exp(-0.5*dxy/a)
      return torch.mean(XX + YY - 2. * XY)
    
    def init_params(self):

        stdv = 1. / math.sqrt(self.embeddings_int.size(1))
        self.embeddings_int.data.uniform_(-stdv, stdv)
        self.embeddings_pop.data.uniform_(-stdv, stdv)
        self.embeddings_soc.data.uniform_(-stdv, stdv)

    def adapt(self, epoch, decay):

        self.int_weight = self.int_weight*decay
        self.pop_weight = self.pop_weight*decay
        self.soc_weight = self.soc_weight*decay

    def bpr_loss(self, p_score, n_score):

        return -torch.mean(torch.log(torch.sigmoid(p_score - n_score)))

    def mask_bpr_loss(self, p_score, n_score, mask):

        return -torch.mean(mask*torch.log(torch.sigmoid(p_score - n_score)))

    def forward(self, user, item_p, item_n, mask,nei_mask, training=True):
        #self.graph = self.graph.to("cpu")
        self.graph = self.graph.to("cuda")

        features_int = [self.embeddings_int]
        h = self.embeddings_int
        for layer in self.layers:
            h = layer(self.graph, h)
            h = F.dropout(h, p=self.dropout, training=training)
            features_int.append(h)

        features_int = torch.stack(features_int, dim=2)
        features_int = torch.mean(features_int, dim=2)

        features_pop = [self.embeddings_pop]
        h = self.embeddings_pop
        for layer in self.layers:
            h = layer(self.graph, h)
            h = F.dropout(h, p=self.dropout, training=training)
            features_pop.append(h)

        features_pop = torch.stack(features_pop, dim=2)
        features_pop = torch.mean(features_pop, dim=2)

        features_soc = [self.embeddings_soc]
        h = self.embeddings_soc
        for layer in self.layers:
            h = layer(self.graph, h)
            h = F.dropout(h, p=self.dropout, training=training)
            features_soc.append(h)

        features_soc = torch.stack(features_soc, dim=2)
        features_soc = torch.mean(features_soc, dim=2)

        item_p = item_p + self.n_user 
        item_n = item_n + self.n_user 

        users_int = features_int[user]
        users_pop = features_pop[user]
        users_soc = features_soc[user]
        items_p_int = features_int[item_p]
        items_p_pop = features_pop[item_p]
        items_p_soc = features_soc[item_p]
        items_n_int = features_int[item_n]
        items_n_pop = features_pop[item_n]
        items_n_soc = features_soc[item_n]

        p_score_int = torch.sum(users_int*items_p_int, 2)
        n_score_int = torch.sum(users_int*items_n_int, 2)

        p_score_pop = torch.sum(users_pop*items_p_pop, 2)
        n_score_pop = torch.sum(users_pop*items_n_pop, 2)

        p_score_soc = torch.sum(users_soc*items_p_soc, 2)
        n_score_soc = torch.sum(users_soc*items_n_soc, 2)

        p_score_total = p_score_int + p_score_pop + p_score_soc
        n_score_total = n_score_int + n_score_pop + n_score_soc
        
        loss_int = self.mask_bpr_loss(p_score_int, n_score_int, mask) + self.mask_bpr_loss(p_score_int, n_score_int, nei_mask)
        loss_soc = self.mask_bpr_loss(n_score_soc, p_score_soc, nei_mask) + self.mask_bpr_loss(p_score_soc, n_score_soc, ~nei_mask)
        loss_pop = self.mask_bpr_loss(n_score_pop, p_score_pop, mask) + self.mask_bpr_loss(p_score_pop, n_score_pop, ~mask)
        loss_total = self.bpr_loss(p_score_total, n_score_total)

        item_all = torch.unique(torch.cat((item_p, item_n)))
        item_int = features_int[item_all]
        item_pop = features_pop[item_all]
        item_soc = features_soc[item_all]
        user_all = torch.unique(user)
        user_int = features_int[user_all]
        user_pop = features_pop[user_all]
        user_soc = features_soc[user_all]
        discrepency_loss = self.criterion_discrepancy(item_int, item_pop) + self.criterion_discrepancy(item_int, item_soc) + self.criterion_discrepancy(item_soc, item_pop) + self.criterion_discrepancy(user_int, user_pop) + self.criterion_discrepancy(user_soc, user_pop) + self.criterion_discrepancy(user_int, user_soc)
        loss = self.int_weight*loss_int + self.pop_weight*loss_pop + self.soc_weight*loss_soc + loss_total - self.dis_pen*discrepency_loss

        return loss

    def get_embeddings(self):

        features_int = [self.embeddings_int]
        h = self.embeddings_int
        for layer in self.layers:
            h = layer(self.graph, h)
            features_int.append(h)

        features_int = torch.stack(features_int, dim=2)
        features_int = torch.mean(features_int, dim=2)

        users_int = features_int[:self.n_user]
        items_int = features_int[self.n_user:]

        features_pop = [self.embeddings_pop]
        h = self.embeddings_pop
        for layer in self.layers:
            h = layer(self.graph, h)
            features_pop.append(h)

        features_pop = torch.stack(features_pop, dim=2)
        features_pop = torch.mean(features_pop, dim=2)
        users_pop = features_pop[:self.n_user]
        items_pop = features_pop[self.n_user:]

        features_soc = [self.embeddings_soc]
        h = self.embeddings_soc
        for layer in self.layers:
            h = layer(self.graph, h)
            features_soc.append(h)

        features_soc = torch.stack(features_soc, dim=2)
        features_soc = torch.mean(features_soc, dim=2)
        users_soc = features_soc[:self.n_user]
        items_soc = features_soc[self.n_user:]

        items = torch.cat((items_int, items_pop,items_soc), 1)
        users = torch.cat((users_int, users_pop,users_soc), 1)

        return items.detach().cpu().numpy().astype('float32'), users.detach().cpu().numpy().astype('float32')

Using backend: pytorch


In [ ]:
from tqdm import tqdm
import torch

f = "/content/drive/MyDrive/Implicit/Ciao/train_adj_DICE.npz"
model = LGNSIDR(7373, 91091, 64,3,0.5,1e-3, 0.1, 0.1,0.1,f)
model = model.cuda()
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/DICE_20_0.001",map_location=torch.device('cpu')))
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/MyModel_MMD_Test_NoNetwork_"+str(0)+"_"+str(1e-3)))
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
min_loss = 1e+5
for epoch in range(0,500):
  running_loss = 0
  total_loss = 0
  for batch_count, sample in enumerate(tqdm(x)):
    user, item_p, item_n, mask, nei_mask = sample
    user = user.cuda()
    item_p = item_p.cuda()
    item_n = item_n.cuda()
    mask = mask.cuda()
    nei_mask = nei_mask.cuda()
    loss = model.forward(user, item_p, item_n, mask,nei_mask)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()
  if(round(total_loss/(batch_count+1),2)<round(min_loss/(batch_count+1),2)):
    min_loss = total_loss
    breakout = 0 
  else:
    breakout+=1
  if(breakout==5):
    break
  if(epoch%3==0):
      file = "/content/drive/MyDrive/Implicit/Ciao/Models/MyModel_MMD_Test_NoNetwork_"+str(epoch)+"_"+str(1e-3)
      torch.save(model.state_dict(),f=file)
      print("Model Saved")
  print("BPR Loss is ", round(total_loss/(batch_count+1),2), " for epoch ", epoch)

/usr/local/lib/python3.7/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


tensor([    0,     1,     2,  ..., 98461, 98462, 98463])


100%|██████████| 354/354 [09:43<00:00,  1.65s/it]


Model Saved
BPR Loss is  0.49  for epoch  0


100%|██████████| 354/354 [09:36<00:00,  1.63s/it]


...

BPR Loss is  0.12  for epoch  11


100%|██████████| 354/354 [09:20<00:00,  1.58s/it]


Model Saved
BPR Loss is  0.11  for epoch  12


 16%|█▌        | 57/354 [01:31<07:56,  1.60s/it]


KeyboardInterrupt: ignored

In [ ]:
from tqdm import tqdm
import torch

f = "/content/drive/MyDrive/Implicit/Epinions/train_adj_DICE.npz"
model = LGNSIDR(37952,119015, 64,2,0.2,1e-3, 0.1, 0.1,0.1,f)
model = model.cuda()
model.graph = model.graph.to("cuda")
model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/Ablation_Test_"+str(9)+"_"+str(1e-3)))
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Ciao/Models/MyModel_MMD_Test_NoNetwork_"+str(12)+"_"+str(1e-3)))
features_int = [model.embeddings_int]
h = model.embeddings_int
for layer in model.layers:
    h = layer(model.graph, h)
    features_int.append(h)

features_int = torch.stack(features_int, dim=2)
features_int = torch.mean(features_int, dim=2)

users_int = features_int[:model.n_user]
items_int = features_int[model.n_user:]

features_pop = [model.embeddings_pop]
h = model.embeddings_pop
for layer in model.layers:
    h = layer(model.graph, h)
    features_pop.append(h)

features_pop = torch.stack(features_pop, dim=2)
features_pop = torch.mean(features_pop, dim=2)
users_pop = features_pop[:model.n_user]
items_pop = features_pop[model.n_user:]

features_soc = [model.embeddings_soc]
h = model.embeddings_soc
for layer in model.layers:
    h = layer(model.graph, h)
    features_soc.append(h)

features_soc = torch.stack(features_soc, dim=2)
features_soc = torch.mean(features_soc, dim=2)
users_soc = features_soc[:model.n_user]
items_soc = features_soc[model.n_user:]

item_emb = torch.cat((items_int,items_soc,items_pop), 1)
user_emb = torch.cat((users_int,users_soc,users_pop), 1)

/usr/local/lib/python3.7/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


tensor([     0,      1,      2,  ..., 156964, 156965, 156966])


In [ ]:
import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/basic_model_without_network"+str(180)+"_"+str(1e-2),pickle_module=dill)['state_dict'])
#path = "/content/drive/MyDrive/Copy of Epinions/testData_Final/"
#path = "/content/drive/MyDrive/Copy of Epinions/testData_Final/ratings_data_test.csv"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data_test.csv"
#files = os.listdir(path)
path = "/content/drive/MyDrive/Implicit/Epinions/"
model.eval()

files = os.listdir(path)
for f in files:
  if("test" in f and f in ["ratings_data_test_implicit1_1_0.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_5_0.csv","ratings_data_test_implicit1_10_0.csv"]):
    if(f=="test.csv"):
      continue
    print(f)  
    test = pd.read_csv(path+f)
    test = test[["userId","productId","rating"]]
    test_negatives = _sample_negative(list(test["productId"].unique()),test)
    a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
    del test_negatives
    test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
    for row in a.itertuples():
      test_users.append(int(row.userId))
      test_items.append(int(row.productId))
      ratings.append(float(row.rating))
      for i in range(len(row.negative_samples)):
          negative_users.append(int(row.userId))
          negative_items.append(int(row.negative_samples[i]))
          negative_ratings.append(float(0))  # negative samples get 0 rating
    del a
    tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
    predr = []
    neg_predr = []
    for i in range(len(tu)):
      user_vec = user_emb[tu[i]]
      item_vec = item_emb[ti[i]]
      score = torch.sigmoid(torch.matmul(user_vec,item_vec.T))
      predr.append(score.cpu().detach().numpy())
    for i in range(len(nu)):
      user_vec = user_emb[nu[i]]
      item_vec = item_emb[ni[i]]
      score = torch.sigmoid(torch.matmul(user_vec,item_vec.T))
      neg_predr.append(score.cpu().detach().numpy())
    subjects = [tu,ti,predr,nu,ni,neg_predr]
    test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
    neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
    # the golden set
    test = pd.DataFrame({'user': test_users,
                        'test_item': test_items,
                        'test_score': test_scores})
    # the full set
    full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                      'item':  np.append(neg_items,test_items),
                      'score':  np.append(neg_scores,test_scores)})
    full = pd.merge(full, test, on=['user'], how='left')
    # rank the items according to the scores for each user
    full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
    full.sort_values(['user', 'rank'], inplace=True)
    subjects = full
    print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
    print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))
    del full

ratings_data_test_implicit1_2_0.csv
Hit Ratio is:  0.18
NDCG Value is:  0.11
ratings_data_test_implicit1_5_0.csv
Hit Ratio is:  0.4
NDCG Value is:  0.24
ratings_data_test_implicit1_10_0.csv
Hit Ratio is:  0.85
NDCG Value is:  0.63
ratings_data_test_implicit1_1_0.csv
Hit Ratio is:  0.13
NDCG Value is:  0.08
